In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/gss.csv')

<ipython-input-2-40bbe69a4d3f>:1: DtypeWarning: Columns (20,50,68,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/gss.csv')


In [5]:
print(df.head(10))

   Unnamed: 0  year  id            wrkstat  hrs1  hrs2 evwork    occ  \
0           1  1972   1  working full time   NaN   NaN    NaN  205.0   
1           2  1972   2            retired   NaN   NaN    yes  441.0   
2           3  1972   3  working part time   NaN   NaN    NaN  270.0   
3           4  1972   4  working full time   NaN   NaN    NaN    1.0   
4           5  1972   5      keeping house   NaN   NaN    yes  385.0   
5           6  1972   6  working full time   NaN   NaN    NaN  281.0   
6           7  1972   7  working full time   NaN   NaN    NaN  522.0   
7           8  1972   8  working full time   NaN   NaN    NaN  314.0   
8           9  1972   9  working part time   NaN   NaN    NaN  912.0   
9          10  1972  10  working full time   NaN   NaN    NaN  984.0   

   prestige        wrkslf  ...  instype02  instype03  instype04  \
0      50.0  someone else  ...        NaN        NaN        NaN   
1      45.0  someone else  ...        NaN        NaN        NaN   
2     